In [1]:
%matplotlib inline
import pandas as pd

In [2]:
compras_df = pd.read_csv('/content/drive/MyDrive/pagos_provedores_monterrey/data/clean_compras.csv')

In [3]:
compras_df.dtypes

,0
C B,int64
BENEFICIARIO,object
RFC,object
FACTURAS,object
CONTRATO,object
CONCEPTO,object
TIPO,object
FDO-#OP,float64
FECHA,object
IMPORTE,float64


## Empezamos con una expansion de datos que tenemos, fecha por ejemplo

In [4]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])

In [5]:
compras_df['DIA'] = compras_df['FECHA'].apply(lambda x: x.day)
compras_df['MES'] = compras_df['FECHA'].apply(lambda x: x.month)
compras_df['AÑO'] = compras_df['FECHA'].apply(lambda x: x.year)

### Proyectemos unas variables contra otras

In [6]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [7]:
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'MUNICIPIO DE LA CIUDAD DE MONTERREY',
 'EDITORA EL SOL, S.A. DE C.V.',
 'MOTO SOLUCIONES MEXICO, S.A. DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.']

In [9]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x])

In [11]:
sheet_mean_compras = compras_df.groupby('SHEETS')['IMPORTE'].mean()

In [13]:
sheet_mean_compras

,IMPORTE
SHEETS,
COMUNIC,9.979830e+04
GTOS VARIOS,1.041125e+06
OBRA PUBLICA Y GV FDOS. ESP,4.422806e+05
SER PROF,5.464488e+05
SERV PERS,5.124681e+04


In [14]:
sheet_mean_compras.to_dict()

{'COMUNIC': 99798.29520107238,
 'GTOS VARIOS': 1041124.8811964287,
 'OBRA PUBLICA Y GV FDOS. ESP': 442280.55925343814,
 'SER PROF': 546448.8149333333,
 'SERV PERS': 51246.81125984252}

In [16]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEETS'].apply(lambda x: sheet_mean_compras[x])

### Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

In [17]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('/content/drive/MyDrive/pagos_provedores_monterrey/data/unique_beneficiarios.csv', index = None)

In [19]:
# Cargar
linkedin_mty = pd.read_csv('/content/drive/MyDrive/pagos_provedores_monterrey/data/linkedin_mty.csv')

In [20]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
  has_site[ele['query']] = True if ele['error'] == True else False

In [26]:
# Si `has_site` es un diccionario y queremos obtener el valor de una clave limpia de espacios dobles
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(
    lambda x: has_site.get(' '.join(x.split()), None)  # Reemplaza todos los espacios múltiples y maneja claves no encontradas
)

In [27]:
compras_df['HAS_LINKEDIN_SITE']

,HAS_LINKEDIN_SITE
0,None
1,True
2,True
3,True
4,None
...,...
2657,None
2658,None
2659,None
2660,None


In [28]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

,count
HAS_LINKEDIN_SITE,
True,1042
False,46


In [29]:
compras_df.to_csv('/content/drive/MyDrive/pagos_provedores_monterrey/data/compras_df_enriched', index = None)